# 개인 과제
## 주피터랩에서 데이터를 입력받으면 3가지 DB에 동시에 CRUD가 되는 시스템을 만들어라
### input을 이용하여 <이름> 홍길동(엔터), <나이> 33(엔터), <주소> 해운대구 (엔터)
### 입력하고, 이름을 고르고 수정하고, 삭제 가능하게 하라 

In [120]:
str = """
1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료
고르세요.
"""

while True:
    print(str)
    inp = input("숫자를 선택하세요")
    
    if inp == '1':
        name = input("이름을 입력하세요")
        age = input('나이를 입력하세요')
        address = input('주소를 입력하세요')
        print(f"입력하신 내용은 이름: {name}, 나이: {age} , 주소: {address}")
        oracle.insert_db(name, int(age), address)
        print("오라클 등록 완료")
        mysql.insert_db(name, int(age), address)
        print("MySQL 등록 완료")
        mongodb.insert_db(name, int(age), address)
        print("MongoDB 등록 완료")
        break
    if inp == '2':
        oracle.select_db()
        print("오라클 읽기 완료")
        mysql.select_db()
        print("MySQL 읽기 완료")
        mongodb.select_db()
        print("MongoDB 읽기 완료")
        break
    if inp == '3':
        print("""
        1. ID
        2. NAME
        3. ADDRESS
        4. AGE
        """)
        f_column = input("찾으려는 항목의 숫자를 입력하세요.")
        if f_column != 1 or 2 or 3 or 4:
            print("no")
            f_column = input("찾으려는 항목의 숫자를 입력하세요.")
        find_value = input("찾으려는 값을 입력하세요.")
        print("""
        1. ID
        2. NAME
        3. ADDRESS
        4. AGE
        """)
        c_column = input("변경할 항목의 숫자를 입력하세요.")
        change = input("변경할 값을 입력하세요.")


        
        oracle.update_db(change, find_value, c_column, f_column)
                       # f"UPDATE PRACTICE0409 SET {c_column} = :change WHERE {f_column} = :find_value"
        print("오라클 수정 완료")
        mysql.update_db()
        # f"UPDATE judb.PRACTICE0409 SET {c_column} = %s WHERE {f_column} = %s"
        print("MySQL 수정 완료")
        mongodb.update_db()
        # collection.update_one({f_column:f_value},{"$set":{c_column:c_value}})
        print("MongoDB 수정 완료")
    if inp == '4':
        oracle.delete_db()
        print("오라클 삭제 완료")
        mysql.delete_db()
        print("MySQL 삭제 완료")
        mongodb.delete_db()
        print("MongoDB 삭제 완료")
    if inp == '5':
        print("종료합니다")
        break


1. 데이터 등록
2. 데이터 읽기
3. 데이터 수정
4. 데이터 삭제
5. 종료
고르세요.



숫자를 선택하세요 3



        1. ID
        2. NAME
        3. ADDRESS
        4. AGE
        


찾으려는 항목의 숫자를 입력하세요. 5


no


찾으려는 항목의 숫자를 입력하세요. 1


KeyboardInterrupt: Interrupted by user

In [121]:
import mysql.connector

class Mysql:
    def __init__(self): 
        try:
            self.conn = mysql.connector.connect(
                host = "mysql",
                user = "root",
                password = "1234"
            )
            
            if self.conn.is_connected():
                print("MySQL 접속 완료")
        except mysql.connector.Error as e:
            print(e)
        
        self.cursor = self.conn.cursor()
    
    def insert_db(self, name, age, address):
        query = "INSERT INTO judb.PRACTICE0409 (NAME, AGE, ADDRESS) VALUES (%s, %s, %s)"
        values = (name, age, address)
        self.cursor.execute(query, values)
        self.conn.commit()
        # print("데이터 삽입 완료")
        
    def select_db(self):
        query = "SELECT NAME, AGE, ADDRESS FROM judb.PRACTICE0409"
        self.cursor.execute(query)
        result = self.cursor.fetchall()
        for r in result:
            print(r)
            
    def update_db(self, change, find_value, c_column, f_column):
        try:
            query = f"UPDATE judb.PRACTICE0409 SET {c_column} = %s WHERE {f_column} = %s"
            values = (change, find_value)
            self.cursor.execute(query, values)
            self.conn.commit()
            # print("ok")
        except Exception as e:
            print(e)
        
    def delete_db(self, column, value):
        try:
            query = f"DELETE FROM judb.PRACTICE0409 WHERE {column} = %s"
            value = (value,)
            self.cursor.execute(query, value)
            self.conn.commit()
        except Exception as e:
            print(e)

mysql = Mysql()

MySQL 접속 완료


In [123]:
from pymongo import MongoClient

class MongoDB:
    client = MongoClient("mongodb://mongodb:27017")

    try:
        print('MongoDB 접속 성공')
    except Exception as e:
        print(e)
        
    db = client["PRACTICE0409"]
    # print(f"{db} DB 사용")
    
    collection = db["PRACTICE0409"]
    # print(f"{collection} 컬렉션 사용")
    def insert_db(name, age, address):
        values = {"NAME":name, "AGE":age, "ADDRESS":address}
        insert_values = collection.insert_one(values)
        # print("ok")
    def select_db():
        result = collection.find({},{"_id":0})
        for r in result:
            print(r)
    def update_db(f_column, f_value, c_column, c_value):
        collection.update_one({f_column:f_value},{"$set":{c_column:c_value}})
    def delete_db(d_column, d_value):
        collection.delete_one({d_column:d_value})

mongodb = MongoDB

MongoDB 접속 성공


In [122]:
import oracledb

class Oracle:
    def __init__(self):
        try:
            self.conn = oracledb.connect(
                user = 'pknu',
                password = '1234',
                dsn = 'oracle/XEPDB1'
            )
            self.cursor = self.conn.cursor()
            print("Oracle 연결 성공")
        except Exception as e:
            print(e)

    def insert_db(self, name, age, address):
        query = "INSERT INTO PRACTICE0409 (NAME, AGE, ADDRESS) VALUES (:name, :age, :address)"
        values = (name, age, address)
        self.cursor.execute(query, values)
        self.conn.commit()
        # print("데이터 삽입 완료")
        
    def select_db(self):
        query = "SELECT NAME, AGE, ADDRESS FROM PRACTICE0409"
        self.cursor.execute(query)
        result = self.cursor.fetchall()
        for r in result:
            print(r)
            
    def update_db(self, change, find_value, c_column, f_column):
        try:
            query = f"UPDATE PRACTICE0409 SET {c_column} = :change WHERE {f_column} = :find_value"
            values = (change, find_value)
            self.cursor.execute(query, values)
            self.conn.commit()
            # print("ok")
        except Exception as e:
            print(e)
            
    def delete_db(self, column, value):
        try:
            query = f"DELETE FROM PRACTICE0409 WHERE {column} = :value"
            value = (value,)
            self.cursor.execute(query, value)
            self.conn.commit()
        except Exception as e:
            print(e)

oracle = Oracle()

Oracle 연결 성공
